In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = '/home/njs4nu/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto', torch_dtype=torch.float16)


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# 指定本地缓存路径
model_path = "/home/njs4nu/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/snapshots/01c7f73d771dfac7d292323805ebc428287df4f9"

# 加载分词器和模型
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map='auto', torch_dtype=torch.float16)

# 推理示例
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors='pt').to('cuda')
outputs = model.generate(**inputs, max_new_tokens=50)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)


Loading checkpoint shards: 100%|██████████| 2/2 [00:18<00:00,  9.48s/it]


Hello, how are you? I’m fine. I hope you are fine too.
In this article, I am going to share a list of the best Linux distributions for gaming.
I have tested and used all of them personally. So, I can say that


In [3]:
import json
import pandas as pd
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "/home/njs4nu/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-hf/snapshots/01c7f73d771dfac7d292323805ebc428287df4f9"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map='auto', torch_dtype=torch.float16)

# 读取 JSON 文件
file_path = '/home/njs4nu/Desktop/nlp_proj/LLaMA-Factory/data/climatex_test.json'
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

num_data = len(data)

# 结果保存路径
output_csv_path = '/home/njs4nu/Desktop/nlp_proj/inference_results_llama2_7B.csv'
os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)

# 如果文件不存在，先写入 CSV 文件的表头
if not os.path.exists(output_csv_path):
    header_df = pd.DataFrame(columns=["Number", "System", "Instruction", "Input", "Expected Output", "Generated Output", "Match"])
    header_df.to_csv(output_csv_path, index=False, encoding="utf-8")

# 遍历每个数据样本进行推理
for i in range(num_data):
    system = data[i]['system']
    instruction = data[i]['instruction']
    input_text = data[i]['input']
    expected_output = data[i]['output']

    # 构建提示信息
    prompt = f"{system}\n{instruction}\n{input_text}"

    # 编码输入文本
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    try:
        # 模型推理生成输出
        outputs = model.generate(**inputs, max_new_tokens=200)
        generated_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    except Exception as e:
        generated_output = f"Error: {str(e)}"

    # 将结果保存到 CSV 文件中
    result_data = {
        "Number": i + 1,
        "System": system,
        "Instruction": instruction,
        "Input": input_text,
        "Expected Output": expected_output,
        "Generated Output": generated_output,
        "Match": expected_output == generated_output
    }

    # 将当前结果转换为 DataFrame 并追加保存到 CSV 文件
    result_df = pd.DataFrame([result_data])
    result_df.to_csv(output_csv_path, mode='a', index=False, header=False, encoding="utf-8")

    # 打印当前处理进度
    print(f"Processed {i + 1}/{num_data}: Saved to CSV")

print(f"All results saved to {output_csv_path}")


Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.07s/it]


Processed 1/300: Saved to CSV


KeyboardInterrupt: 